# Session 18 - Labs Challenge : Pandas

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data_ecommerce.csv', encoding='unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## SOAL :
1. Buat kolom baru bernama "transValue" yang merupakan total nilai transaksi (hasil perkalian antara Quantity dengan UnitPrice)
2. Negara manakah yang memiliki total dan rata2 nilai transaksi paling tinggi? Munculkan Top 5
3. Siapakah customer yang paling sering melakukan transaksi? dan siapa yang paling besar melakukan transaksi? sertakan jumlah dan nilai transaksinya!
4. Buat kolom baru bernama "Cancelation" yang memberikan keterangan apakah transaksi tersebut merupakan transaksi 'cancel' atau 'retur'. apabila nilai qtt negatif maka cancel. kemudian bandingkan jumlah transaksi cancelled & not-cancelled.
5. Munculkan 5 Negara yang paling sering melakukan cancelation? dan jumlah cancelation nya
6. Buat kolom baru bernama 'Year' dan 'Month' dari kolom Invoice date, kemudian lihat :
    - Bulan mana yang menghasilkan revenue paling besar
    - Bulan mana yang terjadi pesanan terbesar
    - Bulan mana yang terjadi cancelation paling tinggi
    - Jumlah Transaksi
7. Buat dataframe baru bernama rfm_df yang berisi :
    - Customer ID (Key) 
    - jumlah hari dari hari terakhir customer tersebut melakukan transaksi, total jumlah transaksi,
    - Total TransValue
    - Jumlah Transaksi

## Nomor 1

In [4]:
df['transValue'] = df['Quantity'] * df['UnitPrice']

## Nomor 2

In [5]:
country_total_avg = df.groupby('Country').agg({'transValue':['sum', 'mean']}).reset_index().droplevel(1, 1)

country_total_avg.columns = ['Country', 'Total Transaksi', 'Rata-rata Transaksi']
# Top fice country
country_total_avg.sort_values(['Total Transaksi', 'Rata-rata Transaksi'], ascending=[False, False]).reset_index(drop=True).head()

,Country,Total Transaksi,Rata-rata Transaksi
0,United Kingdom,8.187806e+06,16.525065
1,Netherlands,2.846615e+05,120.059696
2,EIRE,2.632768e+05,32.122599
3,Germany,2.216982e+05,23.348943
4,France,1.974039e+05,23.069288


## Nomor 3

In [6]:
customers_transaction = df.groupby('CustomerID').agg({'InvoiceNo':'count',
                                                     'transValue':'sum'}).reset_index()
customers_transaction.columns = ['CustomerID', 'Jumlah Transaksi', 'tranValue']
customers_transaction.sort_values('Jumlah Transaksi', ascending=False).reset_index(drop=True)

,CustomerID,Jumlah Transaksi,tranValue
0,17841.0,7983,40340.78
1,14911.0,5903,132572.62
2,14096.0,5128,57120.91
3,12748.0,4642,29072.10
4,14606.0,2782,11713.85
...,...,...,...
4367,14119.0,1,-19.90
4368,13747.0,1,79.60
4369,15590.0,1,-30.19
4370,15562.0,1,134.55


In [7]:
customers_transaction.sort_values('tranValue', ascending=False).reset_index(drop=True)

,CustomerID,Jumlah Transaksi,tranValue
0,14646.0,2085,279489.02
1,18102.0,433,256438.49
2,17450.0,351,187482.17
3,14911.0,5903,132572.62
4,12415.0,778,123725.45
...,...,...,...
4367,12503.0,1,-1126.00
4368,17603.0,15,-1165.30
4369,14213.0,5,-1192.20
4370,15369.0,1,-1592.49


## Nomor 4

In [8]:
df['Cancelation'] = np.where(df['Quantity'] < 0, 'cancel', 'retur')
df['Cancelation'].value_counts()

retur     531285
cancel     10624
Name: Cancelation, dtype: int64

## Nomor 5

In [9]:
df[df['Cancelation'] == 'cancel'].groupby('Country')['Cancelation'].count().sort_values(ascending=False).head().reset_index()

,Country,Cancelation
0,United Kingdom,9192
1,Germany,453
2,EIRE,302
3,France,149
4,USA,112


## Nomor 6

In [10]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month

- Bulan mana yang menghasilkan revenue paling besar

In [11]:
df.groupby(['Year', 'Month'])['transValue'].sum().sort_values(ascending=False)

Year  Month
2011  11       1461756.250
      10       1070704.670
      9        1019687.622
2010  12        748957.020
2011  5         723333.510
      6         691123.120
      3         683267.080
      8         682680.510
      7         681300.111
      1         560000.260
      2         498062.650
      4         493207.121
      12        433668.010
Name: transValue, dtype: float64

- Bulan mana yang terjadi pesanan terbesar

In [12]:
df.groupby(['Year', 'Month']).count()

InvoiceNo  StockCode  Description  Quantity  InvoiceDate  \
Year Month                                                             
2010 12         42481      42481        42356     42481        42481   
2011 1          35147      35147        35033     35147        35147   
     2          27707      27707        27642     27707        27707   
     3          36748      36748        36583     36748        36748   
     4          29916      29916        29690     29916        29916   
     5          37030      37030        36892     37030        37030   
     6          36874      36874        36783     36874        36874   
     7          39518      39518        39363     39518        39518   
     8          35284      35284        35202     35284        35284   
     9          50226      50226        50127     50226        50226   
     10         60742      60742        60638     60742        60742   
     11         84711      84711        84636     84711        84711   
     12         25525      25525        25510     25525        25525   

            UnitPrice  CustomerID  Country  transValue  Cancelation  
Year Month                                                           
2010 12         42481       26850    42481       42481        42481  
2011 1          35147       21912    35147       35147        35147  
     2          27707       20363    27707       27707        27707  
     3          36748       27822    36748       36748        36748  
     4          29916       23198    29916       29916        29916  
     5          37030       28908    37030       37030        37030  
     6          36874       27836    36874       36874        36874  
     7          39518       27502    39518       39518        39518  
     8          35284       27662    35284       35284        35284  
     9          50226       40822    50226       50226        50226  
     10         60742       50695    60742       60742        60742  
     11         84711       65598    84711       84711        84711  
     12         25525       17661    25525       25525        25525

- Bulan mana yang terjadi cancelation paling tinggi
- Jumlah Transaksi

In [13]:
df[df['Cancelation'] == 'cancel'].groupby(['Year', 'Month'])['Cancelation'].count().sort_values(ascending=False).reset_index()

,Year,Month,Cancelation
0,2011,10,1346
1,2011,11,1213
2,2011,9,903
3,2011,3,833
4,2011,6,818
5,2011,7,802
6,2010,12,798
7,2011,1,797
8,2011,4,745
9,2011,5,738


## Nomor 7

In [14]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,transValue,Cancelation,Year,Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,retur,2010,12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,retur,2010,12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,retur,2010,12


In [15]:
date_max = df['InvoiceDate'].max()
date_max

Timestamp('2011-12-09 12:50:00')

In [16]:
rfm_df = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
rfm_df['Selisih'] = date_max - rfm_df['InvoiceDate']
rfm_df

,CustomerID,InvoiceDate,Selisih
0,12346.0,2011-01-18 10:17:00,325 days 02:33:00
1,12347.0,2011-12-07 15:52:00,1 days 20:58:00
2,12348.0,2011-09-25 13:13:00,74 days 23:37:00
3,12349.0,2011-11-21 09:51:00,18 days 02:59:00
4,12350.0,2011-02-02 16:01:00,309 days 20:49:00
...,...,...,...
4367,18280.0,2011-03-07 09:52:00,277 days 02:58:00
4368,18281.0,2011-06-12 10:53:00,180 days 01:57:00
4369,18282.0,2011-12-02 11:43:00,7 days 01:07:00
4370,18283.0,2011-12-06 12:02:00,3 days 00:48:00


In [17]:
rfm_df['Recenvy'] = [(date_max-i).days for i in rfm_df['InvoiceDate']]
rfm_df

,CustomerID,InvoiceDate,Selisih,Recenvy
0,12346.0,2011-01-18 10:17:00,325 days 02:33:00,325
1,12347.0,2011-12-07 15:52:00,1 days 20:58:00,1
2,12348.0,2011-09-25 13:13:00,74 days 23:37:00,74
3,12349.0,2011-11-21 09:51:00,18 days 02:59:00,18
4,12350.0,2011-02-02 16:01:00,309 days 20:49:00,309
...,...,...,...,...
4367,18280.0,2011-03-07 09:52:00,277 days 02:58:00,277
4368,18281.0,2011-06-12 10:53:00,180 days 01:57:00,180
4369,18282.0,2011-12-02 11:43:00,7 days 01:07:00,7
4370,18283.0,2011-12-06 12:02:00,3 days 00:48:00,3


In [18]:
rfm_df.drop('InvoiceDate', axis=1, inplace=True)

In [29]:
rfm_df['Frequency'] = df.groupby('CustomerID')['InvoiceNo'].count().reset_index().drop('CustomerID', axis=1)

In [30]:
rfm_df['Monetary'] = df.groupby('CustomerID').sum()['transValue'].reset_index().drop('CustomerID', axis=1)

In [31]:
rfm_df

,CustomerID,Selisih,Recenvy,Frequancy,Frequency,Monetary
0,12346.0,325 days 02:33:00,325,NaN,2,0.00
1,12347.0,1 days 20:58:00,1,NaN,182,4310.00
2,12348.0,74 days 23:37:00,74,NaN,31,1797.24
3,12349.0,18 days 02:59:00,18,NaN,73,1757.55
4,12350.0,309 days 20:49:00,309,NaN,17,334.40
...,...,...,...,...,...,...
4367,18280.0,277 days 02:58:00,277,NaN,10,180.60
4368,18281.0,180 days 01:57:00,180,NaN,7,80.82
4369,18282.0,7 days 01:07:00,7,NaN,13,176.60
4370,18283.0,3 days 00:48:00,3,NaN,756,2094.88
